***Creator: Changhee Kang***

**Part 2 - Developing predictive model development**

**Note that:** data manupulation and exploration techniques illustrated in this demonstraton does not mean that the audience have to follow the same ways as shown here.

# Device Failure and Maintenance Prediction Model

It is to build a predictive model with diagnoses of telemetry attributes to classify whether maintenance should be performed on devices or not. The column used for prediction is set with the column name, "failure", with binary value 0 for "non-failure" and 1 for "failure". The goal is to minimize false positives and false negatives.

# Assumptions

As there is no meta data that describes the current dataset, assumptions can be applied to the current dataset. The dataset consists of diagnoses of telemetry attributes, so it might be rational that some variables are assumed to consist of *categorical nominal type values* while other variables would consist of *continuous type values*. 

# Roadmap

This part is to demonstrate the predictive model development with an imbalanced dataset. In this part, *'Oversampling'* method will be applied to the current dataset to inflate the number of data point in the minority class. The reason for applying *'Oversampling'* will be explained in detail. When developing the predictive model, a model development without cross-validation and with cross-validation will be compared to see the impact of cross-validation involvments in the predictive model development. Moreover, the right way of applying sampling techniques with cross-validation will also be discussed here. *'Random Oversampling'* is the sampling technique to be adopted to inflate the number of data points in the minority class of the dataset.

# Data Loading

import necessary python modules to load the current dataset into memory.

In [4]:
import pandas as pd
import numpy as np

Load the dataset into memory.

In [5]:
datafile = r'/home/thomas/Downloads/device_failure.csv'
dataset = pd.read_csv(datafile, sep=',', engine='python')

See if the dataset has been loaded correctly. There should be 12 columns meaning 12 variables. 

In [6]:
dataset.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3
